<a href="https://colab.research.google.com/github/trick491/Data-Science-Final-Project/blob/main/data_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pull Together Data**

In [1]:
import pandas as pd
import requests
from io import StringIO

edgar_api = 'https://api.github.com/repos/trick491/Data-Science-Final-Project/contents/data/edgar'
yahoo_api = 'https://api.github.com/repos/trick491/Data-Science-Final-Project/contents/data/yahoo'

edgar_response = requests.get(edgar_api)
yahoo_response = requests.get(yahoo_api)

edgar_df = pd.DataFrame()
yahoo_df = pd.DataFrame()

if edgar_response.status_code == 200:
    edgar_files = edgar_response.json()
    for file in edgar_files:
        if file['name'].endswith('.csv'):
            file_url = file['download_url']
            df_temp = pd.read_csv(file_url)
            edgar_df = pd.concat([edgar_df, df_temp], ignore_index=True)
            print(f"Loaded EDGAR file: {file['name']}")

if yahoo_response.status_code == 200:
    yahoo_files = yahoo_response.json()
    for file in yahoo_files:
        if file['name'].endswith('.csv'):
            file_url = file['download_url']
            df_temp = pd.read_csv(file_url)
            yahoo_df = pd.concat([yahoo_df, df_temp], ignore_index=True)
            print(f"Loaded Yahoo file: {file['name']}")

print(f"\nEDGAR DataFrame shape: {edgar_df.shape}")
print(f"Yahoo DataFrame shape: {yahoo_df.shape}")

Loaded EDGAR file: edgar_1.csv
Loaded EDGAR file: edgar_10.csv
Loaded EDGAR file: edgar_11.csv
Loaded EDGAR file: edgar_12.csv
Loaded EDGAR file: edgar_13.csv
Loaded EDGAR file: edgar_14.csv
Loaded EDGAR file: edgar_15.csv
Loaded EDGAR file: edgar_16.csv
Loaded EDGAR file: edgar_17.csv
Loaded EDGAR file: edgar_18.csv
Loaded EDGAR file: edgar_19.csv
Loaded EDGAR file: edgar_2.csv
Loaded EDGAR file: edgar_20.csv
Loaded EDGAR file: edgar_21.csv
Loaded EDGAR file: edgar_22.csv
Loaded EDGAR file: edgar_23.csv
Loaded EDGAR file: edgar_24.csv
Loaded EDGAR file: edgar_25.csv
Loaded EDGAR file: edgar_26.csv
Loaded EDGAR file: edgar_27.csv
Loaded EDGAR file: edgar_28.csv
Loaded EDGAR file: edgar_29.csv
Loaded EDGAR file: edgar_3.csv
Loaded EDGAR file: edgar_30.csv
Loaded EDGAR file: edgar_31.csv
Loaded EDGAR file: edgar_32.csv
Loaded EDGAR file: edgar_33.csv
Loaded EDGAR file: edgar_34.csv
Loaded EDGAR file: edgar_35.csv
Loaded EDGAR file: edgar_36.csv
Loaded EDGAR file: edgar_37.csv
Loaded EDGA

***